# Part 3: Evaluation (The "Robust" Part)

This is the most important notebook. Without evaluation, you are flying blind. You might change a chunk size and *think* it's better, but you don't *know*.

## Theory: LLM-as-a-Judge
It is hard to measure the quality of text automatically. ROUGE and BLUE scores (from translation) are bad for RAG.
Instead, we use a strong LLM (like GPT-4) to grade the performance of our RAG system. This is the core of the **RAGAS** library.

## Metrics
1. **Faithfulness**: Is the answer derived *only* from the retrieved context? (Hallucination check)
2. **Answer Relevance**: Does the answer actually address the user's question?
3. **Context Precision**: Is the "gold" chunk ranked high in the results?
4. **Context Recall**: Did we retrieve all the information needed to answer?

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

## 1. Creating a Synthetic Test Set (Stroj jako zkoušející)

Představte si, že jste učitel. Máte učebnici (vaše interní dokumenty) a chcete své studenty (váš RAG systém) vyzkoušet, jestli látce opravdu rozumí.
*   **Problém:** Vymýšlet stovky kvalitních otázek a správných odpovědí ručně je nuda, stojí to čas a je to drahé.
*   **Řešení:** Necháme chytřejšího kolegu (GPT-4), aby si učebnici přečetl za nás a testovací otázky automaticky vymyslel.

### Jak to funguje? (Synthetic Data Generation)
Knihovna **Ragas** nepíše jen jednoduché otázky. Snaží se simulovat skutečného, zvídavého uživatele pomocí různých strategií (Evolutions):

1.  **Simple (Faktické):** *"Jaký je limit API requestů?"* (Odpověď leží na jednom místě).
2.  **Reasoning (Uvažovací):** *"Co musím splnit, abych dosáhl na SLA kredit?"* (Odpověď vyžaduje spojení podmínek a logický úsudek).
3.  **Multi-Context (Komplexní):** *"Porovnej podmínky tarifu Basic a Enterprise."* (Odpověď je roztroušena ve více částech dokumentu).

**Výsledek:** Získáme tzv. **Golden Dataset** – sadu otázek (`question`) a správných odpovědí (`ground_truth`), proti které budeme náš systém testovat.

In [7]:
import os
import pandas as pd
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from ragas.testset import TestsetGenerator
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

# 1. Načtení dat
print("--- Načítám dokumenty ---")
loader = DirectoryLoader("../data", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader)
documents = loader.load()
print(f"✅ Načteno {len(documents)} dokumentů.")

# 2. Konfigurace Azure
azure_config = {
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
}

# 3. Inicializace modelů
# Poznámka: Ragas 0.4 používá jeden hlavní LLM pro generování i kritiku v základu
main_llm = AzureChatOpenAI(
    azure_deployment="gpt-4o", 
    temperature=0,
    **azure_config
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002"),
    **azure_config
)

# 4. Vytvoření Generátoru (OPRAVENO)
# Správné názvy argumentů pro Ragas 0.4.1 jsou 'llm' a 'embedding_model'
generator = TestsetGenerator.from_langchain(
    llm=main_llm,
    embedding_model=embeddings
)

# 5. Generování
print("\n--- Generuji syntetické otázky ---")
testset = generator.generate_with_langchain_docs(
    documents, 
    testset_size=5
)

# 6. Výsledek
test_df = testset.to_pandas()
print(f"\n✅ Hotovo. Vygenerováno {len(test_df)} párů.")
display(test_df.head())

--- Načítám dokumenty ---
✅ Načteno 1 dokumentů.

--- Generuji syntetické otázky ---


Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying EmbeddingExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying ThemesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/5 [00:00<?, ?it/s]


✅ Hotovo. Vygenerováno 5 párů.


,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,Wht is us-east-1?,[NebulaDB API Documentation\n\nIntroduction\n\...,us-east-1 is a region code used to filter clus...,Interstellar Systems Engineer,MISSPELLED,SHORT,single_hop_specific_query_synthesizer
1,How do I retrive a list of all actve clusters ...,[NebulaDB API Documentation\n\nIntroduction\n\...,To retrieve a list of all active clusters in N...,Interstellar Systems Engineer,MISSPELLED,MEDIUM,single_hop_specific_query_synthesizer
2,How get list of clusters in mars-north-2?,[NebulaDB API Documentation\n\nIntroduction\n\...,To get a list of clusters in the mars-north-2 ...,Interstellar Systems Engineer,POOR_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
3,Howw do I filter the list of active clusters b...,[NebulaDB API Documentation\n\nIntroduction\n\...,To filter the list of active clusters by the u...,Interstellar Systems Engineer,MISSPELLED,LONG,single_hop_specific_query_synthesizer
4,What information can be retrieved about cluste...,[NebulaDB API Documentation\n\nIntroduction\n\...,"Using the NebulaDB API, you can retrieve a lis...",Interstellar Systems Engineer,PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer


## 2. Evaluate Your Pipeline (Vysvědčení pro robota)

Teď, když máme testovací otázky od "učitele" (syntetický dataset), musíme nechat náš RAG systém, aby na ně odpověděl. Poté jeho odpovědi obodujeme.

### Co vlastně hodnotíme? (Ragas Metriky)
Proci se u RAGu nepoužívá jen jedna známka, ale hodnotíme 4 různé disciplíny:

1.  **Faithfulness (Pravdomluvnost):** Lže model, nebo se drží textu?
    *   *Příklad:* V dokumentu je *"Cena 500 Kč"*. Model řekne *"Cena 600 Kč"*. -> Nízká faithfulness (Model si vymýšlí/halucinuje).
2.  **Answer Relevance (K věci):** Odpověděl model na to, na co jsem se ptal?
    *   *Otázka:* *"Jaké je počasí?"* -> *Odpověď:* *"Dnes je úterý."* -> Nízká relevance (Pravdivé, ale mimo mísu).
3.  **Context Precision (Kvalita vyhledávání):** Byly "zlaté dokumenty" na prvním místě?
    *   Pokud model našel správný dokument, ale až jako 10. v pořadí (utopený v balastu), skóre klesá.
4.  **Context Recall (Úplnost vyhledávání):** Našli jsme všechno, co bylo potřeba?
    *   Pokud k zodpovězení otázky potřebujete 3 různé odstavce, ale systém našel jen 2, skóre klesá.

### Proces Evaluace
Do hodnotící funkce (`evaluate`) musíme poslat 4 věci pro každý testovací případ:
1.  `question`: Co jsme se ptali?
2.  `answer`: Co náš systém odpověděl?
3.  `contexts`: Jaké dokumenty systém při hledání našel?
4.  `ground_truth`: Jaká byla správná odpověď (podle učitele/datasetu)?

In [10]:
from ragas import evaluate
from datasets import Dataset
# OPRAVA IMPORTŮ: answer_relevancy (všimněte si 'cy')
from ragas.metrics import (
    context_precision,
    faithfulness,
    answer_relevancy,  # ZDE BLA CHYBA
    context_recall,
)

print("--- Spouštím Evaluaci ---")

# 1. PŘÍPRAVA DAT (SIMULACE)
data_samples = {
    'question': [
        'What is the API rate limit?', 
        'How do I upgrade to Enterprise?'
    ],
    'answer': [
        'The rate limit is 100 requests per second.',    
        'You can upgrade by contacting sales team.'      
    ],
    'contexts': [
        ['API Rate limit is set to 100 req/sec per token limits.'],  
        ['Contact our sales department for Enterprise licensing and upgrades.'] 
    ],
    'ground_truth': [
        '100 req/sec per token.',    
        'Contact sales to upgrade.'  
    ]
}

# 2. Vytvoření Dataset objektu
dataset = Dataset.from_dict(data_samples)

# 3. Spuštění hodnocení
results = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy, # OPRAVENÝ NÁZEV
        context_recall,
    ],
    llm=critic_llm, 
    embeddings=embeddings
)

# 4. Zobrazení výsledků
print("\n📊 VÝSLEDKY EVALUACE:")
print(results)
df_results = results.to_pandas()
display(df_results.head())

--- Spouštím Evaluaci ---


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.



📊 VÝSLEDKY EVALUACE:
{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9282, 'context_recall': 1.0000}


,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,What is the API rate limit?,[API Rate limit is set to 100 req/sec per toke...,The rate limit is 100 requests per second.,100 req/sec per token.,1.0,1.0,0.941238,1.0
1,How do I upgrade to Enterprise?,[Contact our sales department for Enterprise l...,You can upgrade by contacting sales team.,Contact sales to upgrade.,1.0,1.0,0.915258,1.0
